In [2]:
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
#%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images


Using TensorFlow backend.
/Users/nathanhutchison/anaconda3/envs/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [12]:
#Getting the data and preprocessing the data

folderName = "allFilmFrames"

data = pd.read_csv(folderName + "/filmLabelMapping.csv")

data.head()


imgArr = []
for img_name in data.Image_ID:
    img = plt.imread("" + folderName + "/" + img_name)
    imgArr.append(img)

imgArr = np.array(imgArr)

y = data.Class
dummy_y = np_utils.to_categorical(y)

image = []
for i in range(0,imgArr.shape[0]):
    a = resize(imgArr[i], preserve_range = True, output_shape = (224, 224)).astype(int)
    image.append(a)

imgArr = np.array(image)

from keras.applications.vgg16 import preprocess_input

imgArr = preprocess_input(imgArr, mode='tf')

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(imgArr, dummy_y, test_size = 0.3, random_state = 42)

from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout
base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (224,224,3))

X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape



/Users/nathanhutchison/anaconda3/envs/tensorflow/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/nathanhutchison/anaconda3/envs/tensorflow/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


((1474, 7, 7, 512), (633, 7, 7, 512))

In [13]:


X_train = X_train.reshape(1474, 7*7*512)
X_valid = X_valid.reshape(633, 7*7*512)

X_train.shape, X_valid.shape

train = X_train/X_train.max()
X_valid = X_valid/X_train.max()



In [14]:
#Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))  #input layer
model.add(Dense(units = 1024, activation = 'sigmoid')) # hidden layer
model.add(Dense(3, activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3075      
Total params: 25,694,211
Trainable params: 25,694,211
Non-trainable params: 0
_________________________________________________________________


In [15]:
#Compiling the model

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [17]:
#Training the model
model.fit(train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Train on 1474 samples, validate on 633 samples
Epoch 1/10
1474/1474 [==============================] - 15s 10ms/step - loss: 0.0053 - acc: 0.9993 - val_loss: 0.0102 - val_acc: 0.9984
Epoch 2/10
1474/1474 [==============================] - 15s 10ms/step - loss: 0.0035 - acc: 0.9986 - val_loss: 0.0071 - val_acc: 0.9984
Epoch 3/10
1474/1474 [==============================] - 15s 10ms/step - loss: 0.0033 - acc: 0.9993 - val_loss: 0.0046 - val_acc: 0.9984
Epoch 4/10
1474/1474 [==============================] - 15s 10ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.0055 - val_acc: 0.9968
Epoch 5/10
1474/1474 [==============================] - 15s 10ms/step - loss: 0.0035 - acc: 0.9993 - val_loss: 0.0068 - val_acc: 0.9984
Epoch 6/10
1474/1474 [==============================] - 15s 10ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0035 - val_acc: 0.9968
Epoch 7/10
1474/1474 [==============================] - 15s 10ms/step - loss: 8.2069e-04 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 0.99